<a href="https://colab.research.google.com/github/dondonrocket/kokudo/blob/%E3%83%86%E3%82%B9%E3%83%88/hasegawa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
import geopandas as gpd
import lightgbm as lgb

from shapely.geometry import Point
from sklearn.model_selection import train_test_split

In [15]:
# =========================
# 1. train / test 読み込み
# =========================

train = pd.read_csv(
    "/content/train.csv",
    encoding="shift_jis",
    encoding_errors="replace",
    low_memory=False
)

test = pd.read_csv(
    "/content/test.csv",
    encoding="shift_jis",
    encoding_errors="replace",
    low_memory=False
)




In [23]:
print(train_gdf.columns)


Index(['target_ym', 'money_room', 'building_id', 'building_status',
       'building_create_date', 'building_modify_date', 'building_type',
       'building_name', 'building_name_ruby', 'homes_building_name',
       ...
       'station_passengers_500m_max_log', 'station_passengers_500m_mean_log',
       'station_passengers_1000m_sum', 'station_passengers_1000m_max',
       'station_passengers_1000m_mean', 'station_passengers_1000m_sum_log',
       'station_passengers_1000m_max_log', 'station_passengers_1000m_mean_log',
       'age', 'geometry'],
      dtype='object', length=163)


In [16]:
# =========================
# 2. 駅乗降客数（2019年）
# =========================
df = pd.read_csv("S12-24_NumberOfPassengers_utf8.csv")

df_2019 = df[
    (df["S12_039"] == 1) &  # データ有
    (df["S12_038"] == 1)    # 重複なし
]

station_2019 = (
    df_2019
    .groupby("S12_001c", as_index=False)
    .agg(passengers_2019=("S12_041", "sum"))
    .rename(columns={"S12_001c": "station_code"})
)

In [17]:
# =========================
# 3. 駅ポイント（lon / lat）
# =========================

station_point_gdf = gpd.read_file("/content/S12-24_NumberOfPassengers.geojson")

# 型合わせ（merge対策）
station_point_gdf["S12_001c"] = station_point_gdf["S12_001c"].astype(str)
station_2019["station_code"] = station_2019["station_code"].astype(str)

# 乗降客数と結合
stations = station_point_gdf.merge(
    station_2019,
    left_on="S12_001c",
    right_on="station_code",
    how="left"
)

#先にメートル系へ変換
stations = stations.to_crs(epsg=3857)

# centroid を Point として使う
stations["geometry"] = stations.geometry.centroid

# これで stations は「駅ポイントの GeoDataFrame」
stations_gdf = stations[[
    "S12_001c",
    "passengers_2019",
    "geometry"
]].copy()

stations_gdf.crs = "EPSG:3857"


In [18]:
def add_station_features(df, stations_gdf, radius=500):

    df = df.copy()

    # 物件をGeoDataFrame化
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["lon"], df["lat"]),
        crs="EPSG:4326"
    ).to_crs(epsg=3857)

    # 空間結合
    joined = gpd.sjoin(
        gdf,
        stations_gdf,
        how="left",
        predicate="dwithin",
        distance=radius
    )


    # 集計
    feat = (
        joined
        .groupby("building_id", as_index=False)
        .agg(
            **{
                f"station_passengers_{radius}m_sum":   ("passengers_2019", "sum"),
                f"station_passengers_{radius}m_max":   ("passengers_2019", "max"),
                f"station_passengers_{radius}m_mean":  ("passengers_2019", "mean"),
            }
        )
    )

    # マージ
    df = df.merge(feat, on="building_id", how="left")

    # 欠損処理 + log
    for col in feat.columns:
        if col != "building_id":
            df[col] = df[col].fillna(0)
            df[col + "_log"] = np.log1p(df[col])

    return df


In [19]:
# =========================
# 5. train / test に付与
# =========================
# 500m
train = add_station_features(train, stations_gdf, radius=500)
test  = add_station_features(test, stations_gdf, radius=500)

# 1000m（追加）
train = add_station_features(train, stations_gdf, radius=1000)
test  = add_station_features(test, stations_gdf, radius=1000)

[c for c in train.columns if "station_passengers" in c]

['station_passengers_500m_sum',
 'station_passengers_500m_max',
 'station_passengers_500m_mean',
 'station_passengers_500m_sum_log',
 'station_passengers_500m_max_log',
 'station_passengers_500m_mean_log',
 'station_passengers_1000m_sum',
 'station_passengers_1000m_max',
 'station_passengers_1000m_mean',
 'station_passengers_1000m_sum_log',
 'station_passengers_1000m_max_log',
 'station_passengers_1000m_mean_log']

In [20]:
# =========================
# 6. 築年 → 築年数
# =========================
for df in [train, test]:
    df["building_create_date"] = pd.to_numeric(
        df["building_create_date"], errors="coerce"
    )
    df["age"] = (2023 - df["building_create_date"]).clip(0, 100)


In [21]:
# =========================
# 7.地価公示データ読み込み
# =========================
land_gdf = gpd.read_file("/content/L01-23.geojson")
land_gdf = land_gdf.to_crs(epsg=4326)  # 緯度経度に統一

# train/testをGeoDataFrame化
for df in [train, test]:
    df["geometry"] = gpd.points_from_xy(df["lon"], df["lat"])
train_gdf = gpd.GeoDataFrame(train, geometry="geometry", crs="EPSG:4326")
test_gdf  = gpd.GeoDataFrame(test, geometry="geometry", crs="EPSG:4326")

# 距離計算用に平面直角座標系に変換（メートル単位）
train_gdf = train_gdf.to_crs(epsg=6668)
test_gdf  = test_gdf.to_crs(epsg=6668)
land_gdf  = land_gdf.to_crs(epsg=6668)

In [24]:
# train_gdf に住所紐付け
 train_gdf = train_gdf.merge(
     land_gdf[['L01_024', 'L01_006']],
     left_on='address',
     right_on='L01_024',
     how='left'
 )
 train_gdf = train_gdf.rename(columns={'L01_006': 'land_price'})

# test_gdf に住所紐付け
 test_gdf = test_gdf.merge(
     land_gdf[['L01_024', 'L01_006']],
     left_on='address',
     right_on='L01_024',
     how='left'
 )
 test_gdf = test_gdf.rename(columns={'L01_006': 'land_price'})

In [ ]:
# =========================
# 緯度経度最近傍（距離減衰）
# =========================
def weighted_nearest_price(pt, land_gdf, decay=500):
    """
    緯度経度で最近傍の地価を距離減衰付きで算出
    pt: shapely Point
    land_gdf: GeoDataFrame（geometry, L01_006列）
    decay: 距離減衰パラメータ（メートル単位）
    """
    distances = land_gdf.geometry.distance(pt)
    if (distances == 0).any():  # 完全一致の座標があればその価格
        return land_gdf.loc[distances.idxmin(), 'L01_006']
    weights = np.exp(-distances / decay)
    return np.sum(weights * land_gdf['L01_006']) / np.sum(weights)


mask = train_gdf['land_price'].isna()  # すでに merge で追加した列を使う
train_gdf.loc[mask, 'nearest_land_price'] = train_gdf.loc[mask, 'geometry'].apply(
    lambda pt: weighted_nearest_price(pt, land_gdf)
)

mask = test_gdf['land_price'].isna()
test_gdf.loc[mask, 'nearest_land_price'] = test_gdf.loc[mask, 'geometry'].apply(
    lambda pt: weighted_nearest_price(pt, land_gdf)
)



In [ ]:
# =========================
# 市区町村平均価格
# =========================
# L01_023 は市区町村コード列と仮定
city_prices = land_gdf.groupby('L01_023')['L01_006'].mean().reset_index()
city_prices.rename(columns={'L01_006':'city_avg_price'}, inplace=True)

# train/test に merge
train_gdf = train_gdf.merge(city_prices, left_on='city', right_on='L01_023', how='left')
test_gdf  = test_gdf.merge(city_prices,  left_on='city', right_on='L01_023', how='left')

# =========================
# 最終地価特徴量
# =========================
train_gdf['final_land_price'] = train_gdf['land_price'] \
    .combine_first(train_gdf['nearest_land_price']) \
    .combine_first(train_gdf['city_avg_price'])

test_gdf['final_land_price'] = test_gdf['land_price'] \
    .combine_first(test_gdf['nearest_land_price']) \
    .combine_first(test_gdf['city_avg_price'])


In [ ]:
# =========================
# train/testに反映
# =========================
# train/test に反映
train['final_land_price'] = train_gdf['final_land_price']
test['final_land_price']  = test_gdf['final_land_price']

In [ ]:
# =========================
# 8. 学習データ作成
# =========================
y = np.log1p(train["money_room"])

all_data = pd.concat([
    train.drop(columns=["money_room"]),
    test
])

features = all_data.select_dtypes(include=[np.number]).columns.tolist()
features.remove("id")
features.append('final_land_price')

X = train[features]
X_test = test[features]


In [ ]:
# =========================
# 9. LightGBM
# =========================
model = lgb.LGBMRegressor(
    n_estimators=2000,
    learning_rate=0.03,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# 分割
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 学習
model.fit(
    X_train,
    y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric="mape",
)
#model.fit(X, y)

In [ ]:
# =========================
# 10. 予測・提出
# =========================
pred = np.expm1(model.predict(X_test))

submit = pd.DataFrame({
    "id": test["id"],
    "money_room": pred
})

submit.to_csv("submit.csv", index=False, header=False)
print("submit.csv を出力しました")

